# **Instalación de Librerias**

In [1]:
#@title Instalación de Librerias
# En colab ejecutar siempre porque cada cierto tiempo borra las instancias donde 
# se ejecuta Jupyter 

# Por defecto en colab estas librerias vienen instaladas, solo descomentar si 
# se requiere instalarlo local
#!pip install pandas
#!pip install matplotlib
#!pip install plotly
#!pip install requests

# Librerias extras necesarias
!pip install gitpython
!pip install geopandas
!pip install contextily

     |████████████████████████████████| 163kB 5.4MB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
     |████████████████████████████████| 972kB 5.5MB/s 
     |████████████████████████████████| 14.7MB 303kB/s 
     |████████████████████████████████| 10.9MB 42.5MB/s 
     |████████████████████████████████| 18.2MB 1.3MB/s 


# **Importación de Librerias y Parámetros Globales**

In [2]:
#@title Importación de Librerias
import git
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import matplotlib.image as image
import geopandas
import contextily as ctx
import os


# Matplotlib
plt.rcParams.update({'font.size': 14}) # fontsize base

# Poblacion regional segun censo 2017
poblacion = {
             "Arica y Parinacota": 226068,
             "Tarapacá": 330558,
             "Antofagasta": 607534,
             "Atacama": 286168,
             "Coquimbo": 757586,
             "Valparaiso": 1815902,
             "Metropolitana": 7112808,
             "O'Higgins": 914555,
             "Maule": 1044950,
             "Ñuble": 480609,
             "Biobio": 1556805,
             "La Araucanía": 957224,
             "Los Ríos": 384837,
             "Los Lagos": 828708,
             "Aysén": 103158,
             "Magallanes": 166533}

# **Importación Repositorio de Datos Ministerio de Ciencias**

*   Diccionario Datos COVID19 Ministerio de Ciencia: [http://www.minciencia.gob.cl/covid19](http://www.minciencia.gob.cl/covid19)
*   Link a GitHub del Ministerio de Ciencia: [https://github.com/MinCiencia/Datos-COVID19/](https://github.com/MinCiencia/Datos-COVID19/)
*   Link a la licencia que otorga el Ministerio de Ciencia sobre las bases de datos incluidas en GitHub: [http://www.minciencia.gob.cl/sites/default/files/1771596.pdf](http://www.minciencia.gob.cl/sites/default/files/1771596.pdf)


In [3]:
#@title Clonar repositorios Ministerio de Ciencias
# clonar o actualizar repositorio
print("Repo de ministerio de ciencias...")
try:
  g = git.Git("Datos-COVID19")
  print(g.pull())
except:
  print("Cloning Data")
  git.Git("./").clone("https://github.com/MinCiencia/Datos-COVID19.git")
  print("done")

Repo de ministerio de ciencias...
Cloning Data
done


# **Procesamiento Datos**

In [4]:
path = r'/content/Datos-COVID19/output/producto4/'
all_files = []
for file in os.listdir(path):
    if file.endswith(".csv"):
        all_files.append(os.path.join(path, file))

all_files.sort()

li = []
nombre_columnas = []

for filename in all_files:
    fecha = filename[40:50]
    df = pd.read_csv(filename, index_col=None, header=0)
    df['Fecha'] = fecha 
    nombre_columnas.append(fecha)
    li.append(df)

data = pd.concat(li, axis=0, ignore_index=True)

filtrado = data.fillna(0)

filtrado['totaltotal'] =  filtrado[' Casos nuevos'] \
                        + filtrado['Casos  nuevos  totales']\
                        + filtrado['Casos  nuevos'] \
                        + filtrado['Casos nuevos totales']\
                        + filtrado['Casos nuevos']\
                        + filtrado['Casos nuevos totales ']

# Elimna las filas con la palabra "Total"
filtrado = filtrado[~filtrado['Region'].isin(['Total'])]
filtrado = filtrado[~filtrado['Region'].isin([0])]

# Se listan los valores duplicados para identificarlos
#lista = list(set(filtrado['Region']))
#lista.remove(0)
#lista.sort()
#print(lista)

# Estandarizan los Nombres
diccionario = {"Araucania": "Araucanía", "Arica  y  Parinacota": "Arica y Parinacota", "Arica y Paricota": "Arica y Parinacota", "Arica y Parinacota": "Arica y Parinacota", \
               "Aysen": "Aysén","Biobio": "Biobío","Los  Lagos": "Los Lagos", "Los  Rios": "Los Ríos", "Los Rios": "Los Ríos", "Metropolita": "Metropolitana", "O’Higgins": "O'Higgins", \
               "Nuble": "Ñuble", "Tarapaca": "Tarapacá", "Valparaiso": "Valparaíso"}
data_casos_nuevos = filtrado.replace({"Region": diccionario})

print(data_casos_nuevos.head())
print(data_casos_nuevos.dtypes)



               Region   Casos nuevos  ...  Casos activos probables  totaltotal
0  Arica y Parinacota            0.0  ...                      0.0         0.0
1            Tarapacá            0.0  ...                      0.0         0.0
2         Antofagasta            0.0  ...                      0.0         0.0
3             Atacama            0.0  ...                      0.0         0.0
4            Coquimbo            0.0  ...                      0.0         0.0

[5 rows x 41 columns]
Region                            object
 Casos nuevos                    float64
 Casos totales                   float64
 Casos recuperados               float64
Fecha                             object
Casos nuevos                     float64
Casos totales                    float64
Fallecidos                       float64
Casos  nuevos                    float64
Casos  totales                   float64
 Casos fallecidos                float64
Región                            object
%  Casos  t